# Basic example

In this tutorial our goals are to learn:
- How to use `bartiq` to implement a quantum algorithm from a paper
- How to obtain resource estimates for that algorithm
- What are the most important concepts used in `bartiq`

<div class="alert alert-block alert-info admonition note"> <p class="admonition-title"><b>NOTE:</b></p>

This tutorial, as well as all the other tutorials, has been written as a jupyter notebook.
If you're reading it online, you can either keep reading, or go to `docs/tutorials` and play with it in a more interactive way!

</div>

Before we start implementing some real algorithms, let's consider the following simple routine:

TODO: picture from QAHLO

In `bartiq` we call our main object `Routine` – both the whole algorithm here, as well as each operation will be a `Routine`. 
So what do we know about the routines from the picture above?

- Our main routine is called "My algorithm"
- It consists of two subroutines: "a" and "b".
- It takes in a register of size "N".

How to express it in `bartiq`? 
We do that using `QREF` format (TODO: link) – it's a format for expressing algorithms that we developed with running QREs in mind. So let's write our first routine:

In [1]:
my_algorithm = {
    "name": "my_algorithm",
    "type": None,
    "ports": {
        "in": {
            "name": "in",
            "direction": "input",
            "size": "N"
        },
        "out": {
            "name": "out",
            "direction": "output",
            "size": None
        }
    },
}

What do we have here?

- `name`: name of the routine
- `type`: in this case we don't define the type, but in more complex algorithms you might want to add types, such as "basic_gate" or "comparator".
- `ports`: ports define the interface of the routine. The size of the input port is equal to `N` and in general, we won't know the size of the output port until we perform the compilation.

What are we missing? Children.
Before we add them, to the main routine, we need to define them.

In [2]:
routine_a = {
            "name": "a",
            "type": None,
            "ports": {
                "in": {
                    "name": "in",
                    "direction": "input",
                    "size": "N_a"
                },
                "out": {
                    "name": "out",
                    "direction": "output",
                    "size": "N_a"
                }
            },
        }

In [3]:
routine_b = {
            "name": "b",
            "type": None,
            "ports": {
                "in": {
                    "name": "in",
                    "direction": "input",
                    "size": "N_b"
                },
                "out": {
                    "name": "out",
                    "direction": "output",
                    "size": "N_b"
                }
            },
        }

This is nice, but if we actually want to run some resource estimation here, we will need to know how much each subroutine costs.
In fault-tolerant algorithm the resource that people usually care about is how many T gates do you need to implement an algorithm. Therefore, let's say that `a` costs `2*N_a` T gates and `b` costs `N_b*ceil(log_2(N_b))` T gates. This will require adding two new fields to our dictionary:

In [4]:
routine_a["resources"] = {
                "T_gates": {
                    "name": "T_gates",
                    "type": "additive",
                    "value": "2*N_a",
                }
            }

routine_b["resources"] = {
                "T_gates": {
                    "name": "T_gates",
                    "type": "additive",
                    "value": "N_b*ceil(log_2(N_b))"
                }
            }

routine_a["input_params"] = ["N_a"]
routine_b["input_params"] = ["N_b"]

As you can see we added two new fields to our dictionaries:

- `input_params`: define what variables are used to define resources and register sizes.
- `resources`: what resources are needed for a particular subroutine. As you can see resources have the following fields:
    - `name`: name of the resource
    - `type`: `bartiq` allows for the following types: `additive`, `multiplicative`, `qubits` and `other`.
    - `value`: expression (or numeric value) defining the cost.


Now that `routine_a` and `routine_b` are complete, we can add the missing components to `my_algorithm`:

In [5]:
my_algorithm["children"] = {"a": routine_a, "b": routine_b}
my_algorithm["connections"] = [
        {"source": "in","target": "a.in"},
        {"source": "a.out","target": "b.in"},
        {"source": "b.out","target": "out"}
    ]
my_algorithm["linked_params"] = {
        "N": [("a","N_a"), ("b","N_b")]
    }

What new do we have here?
We got:
- `connections` – how ports are connected. Each connection has source and target.
- `children`: specifies what subroutines does a particular routine consist of.
- `linked_params`: tells us how the parameters of the parent are linked to the parameters of children. In this case, it tells us that parameter `N` from this routine translates to `N_a` in child `a` and `N_b` in child `b`.

Now we can translate our algorithm into a proper `bartiq` routine and see what's the total cost of `my_algorithm:

<div class="alert alert-block alert-info admonition note"> <p class="admonition-title"><b>NOTE:</b></p>

- You might be wondering, why `my_algorithm` didn't need `input_params`? That's because it `N` does not show up in any expressions – we don't have resources defined here. It does show up in the ports though, so we still need to add it to `linked_params`.
- Unfortunately at the moment register sizes need to be single variables – fortunately, as we'll see later, there's a way around it using `local_variables`.

</div>

## Compilation

Below you can find depiction of `my_algorithm`.
![title](../images/basic_uncompiled.png)

We can create `bartiq` `Routine` out of it by simply running:

In [6]:
from bartiq import Routine
uncompiled_routine = Routine(**my_algorithm)

What does "uncompiled" means here?

It means that all the costs and register sizes are expressed using local variables (as in the picture above). What does it mean? Look at this:

In [7]:
uncompiled_routine.children["a"].resources

{'T_gates': <Resource name="T_gates" value="2*N_a">}

The cost of "a" is still expressed in terms of its own "local" variable, `N_a`. Information that we included in `linked_params` has not yet been propagated into "a".

We also don't know yet what's the size of the output port:

In [8]:
uncompiled_routine.ports["out"]

Port(name='out', parent=<Routine name="my_algorithm">, direction='output', size=None, meta={})

Most importantly, we don't know what is the total cost of the algorithm:

In [9]:
uncompiled_routine.resources

{}

So what we want to do, is to get to the following picture (we highlighted in green what changed from the previous one):
![title](../images/basic_compiled.png)

We do this with the following command:

In [10]:
from bartiq import Routine, compile_routine
compiled_routine = compile_routine(uncompiled_routine)

Now if we check the same fields of our `compiled_routine` object, we'll see that everything is expressed in terms of the 

In [11]:
print("T gates for A:", compiled_routine.children["a"].resources["T_gates"].value)
print("Output size:", compiled_routine.ports["out"].size)
print("Total T gates:", compiled_routine.resources["T_gates"].value)

T gates for A: 2*N
Output size: N
Total T gates: N*ceiling(log2(N)) + 2*N


## Evaluation

Now it would be good to know what is the cost when we subsitute some numbers. We can do this using `evaluate` method:

In [12]:
from bartiq import evaluate

for N in range(6, 16, 2):
    assignments = [f"N={N}"]
    evaluated_routine = evaluate(compiled_routine, assignments)
    print(f"N = {N}, total T gates:", evaluated_routine.resources["T_gates"].value)

N = 6, total T gates: 30
N = 8, total T gates: 40
N = 10, total T gates: 60
N = 12, total T gates: 72
N = 14, total T gates: 84


<div class="alert alert-block alert-info admonition note"> <p class="admonition-title"><b>NOTE:</b></p>

- You might wonder – if we have `Routine` class, why do we define everything as dictionaries and do `Routine(**my_dict)` at the very end? We cover that in more details in TODO, but the short answer is, that created `Routine` objects by hand might lead to some very hard to trace bugs, so it's our recommendation to always define them as dictionaries.

</div>

## Summary
Let's sum up what we covered in this tutorial:

- How to construct a simple algorithm in `bartiq`
- How to perform compile an estimate
- How to evaluate an estimate

In the next tutorial we'll cover how to implement a more complex algorithm from a paper – you can find the link [here](TODO).